In [6]:
import os

from dotenv import load_dotenv
from etg import GuestRoom

load_dotenv()

# ETG API Credentials
ETG_KEY_ID = os.environ["ETG_KEY_ID"]
ETG_API_KEY = os.environ["ETG_API_KEY"]
ETG_REQUEST_TIMEOUT = 30.0

# Search Parameters
CITY = "Москва"

CHECKIN_DATE = "2026-02-02"
CHECKOUT_DATE = "2026-02-04"

CURRENCY = "RUB"
LANGUAGE = "ru"
RESIDENCY = "RU"

GUESTS: list[GuestRoom] = [{"adults": 2, "children": [4, 2]}]
LIMIT = 1000

# User preferences for AI
USER_PREFERENCES = "Обязательно две комнаты и две кровати. Хорошие отзывы. Чистота"

# Filters
MIN_PRICE: float | None = 3000.0  # None = no minimum
MAX_PRICE: float | None = 20000.0  # None = no maximum

In [7]:
from etg import ETGClient, GuestRoom, Hotel, HotelContent
from utils import ostrovok_url

client = ETGClient(ETG_KEY_ID, ETG_API_KEY, timeout=ETG_REQUEST_TIMEOUT)

In [8]:
async def find_region_id(client: ETGClient, city_name: str, language: str) -> int | None:
    """Find region ID for a city name."""
    print(f"Looking up region ID for '{city_name}'...")
    regions = await client.suggest_region(city_name, language)

    if not regions:
        print(f"  No regions found for '{city_name}'")
        return None

    # Only accept City type
    for region in regions:
        if region["type"] == "City":
            region_id = region["id"]
            print(f"  Found: {region['name']} ({region.get('country_code', '')}), region_id={region_id}")
            return region_id

    # No city found - show available options
    print(f"  No city found. Available regions:")
    for r in regions[:5]:
        print(f"    - {r['name']} (type: {r['type']}, id: {r['id']})")

    return None

In [9]:
# Find region by city name.
# ETG API requires region_id for hotel search, so we first
# lookup the region ID via suggest_region by city name.
region_id = await find_region_id(client, CITY, LANGUAGE)
if not region_id:
    raise ValueError(f"Could not find region for '{CITY}'")

print(f"\nSearching hotels in {CITY}...")
print(f"  Dates: {CHECKIN_DATE} to {CHECKOUT_DATE}")
print(f"  Currency: {CURRENCY}, Limit: {LIMIT}")

Looking up region ID for 'Москва'...
  Found: Москва (RU), region_id=2395

Searching hotels in Москва...
  Dates: 2026-02-02 to 2026-02-04
  Currency: RUB, Limit: 1000


In [10]:
import pandas as pd

from services import search_hotels

# Search available hotels in the region with given parameters.
# Filters by price range if MIN_PRICE/MAX_PRICE are set.
# Returns short hotel info: id, hid, and rates (room name, price, meal).
# Full content (name, address, amenities) is fetched separately via hotel content API.
search_result = await search_hotels(
    client=client,
    region_id=region_id,
    checkin=CHECKIN_DATE,
    checkout=CHECKOUT_DATE,
    residency=RESIDENCY,
    guests=GUESTS,
    currency=CURRENCY,
    language=LANGUAGE,
    hotels_limit=LIMIT,
    min_price=MIN_PRICE,
    max_price=MAX_PRICE,
)

hotels = search_result["hotels"]
total_available = search_result["total_available"]
total_after_filter = search_result["total_after_filter"]

In [11]:
if not hotels:
    print("No hotels found for the given criteria.")
    df_hotels = pd.DataFrame()
else:
    print(f"Found {total_after_filter} hotels after price filter (total available: {total_available})\n")

    # Create DataFrame
    hotels_data = []
    for hotel in hotels:
        rates = hotel.get("rates", [])
        if rates:
            first_rate = rates[0]
            payment_types = first_rate.get("payment_options", {}).get("payment_types", [])
            price = float(payment_types[0].get("show_amount", 0)) if payment_types else None
            room_name = first_rate.get("room_name", "")
            meal = first_rate.get("meal", "")
        else:
            price = None
            room_name = ""
            meal = ""

        hotels_data.append({
            "hotel_id": hotel["id"],
            "hid": hotel["hid"],
            "price": price,
            "currency": CURRENCY,
            "room": room_name[:40] if room_name else "",
            "meal": meal,
        })

    df_hotels = pd.DataFrame(hotels_data)
    df_hotels = df_hotels.sort_values("price", ascending=True).reset_index(drop=True)
    df_hotels.index += 1  # Start from 1

    # Display settings
    pd.set_option("display.max_colwidth", 50)
    pd.set_option("display.max_rows", 100)

df_hotels

Found 300 hotels after price filter (total available: 329)



,hotel_id,hid,price,currency,room,meal
1,na_baumanskoy_hotel,10613383,6074.0,RUB,Четырёхместный номер семейный Эконом-кла,nomeal
2,travel_inn_krasnyie_vorota,8877227,6951.0,RUB,Семейный номер Economy (общая ванная ком,nomeal
3,apartamentyi_sadovoe_koltso_izumrudnaya,8510363,6970.0,RUB,Апартаменты (питание для детей не включе,nomeal
4,zvezda_hotel_6,10437016,7000.0,RUB,Трёхместный номер Comfort (дополнительна,nomeal
5,minigostinitsa_brusnika_polezhaevskaya,8551959,7149.0,RUB,Двухместный номер Комфорт Плюс (двуспаль,nomeal
...,...,...,...,...,...,...
296,moscow_marriott_hotel_novy_arbat,8139930,38000.0,RUB,Двухместный номер Делюкс (2 отдельные кр,nomeal
297,russkie_sezonyi,8848163,38106.0,RUB,Двухместный люкс with Sofa Superior Манс,nomeal
298,hampton_by_hilton_moscow_rogozhsky_val,9770069,38522.0,RUB,Номер с диваном Семейный с кроватью King,breakfast
299,grand_revival_hotel,10004948,38836.0,RUB,Четырёхместный номер Большой Улучшенный,nomeal


In [12]:
from services import batch_get_content

hotel_hids = [h["hid"] for h in hotels]

print(f"[batch_get_content_start] Загрузка контента для {len(hotel_hids)} отелей...")
content_result = await batch_get_content(client, hotel_hids, LANGUAGE)
content_map = content_result["content"]
print(
    f"[batch_get_content_done] Загружен контент для {content_result['total_loaded']} "
    f"из {content_result['total_requested']} отелей ({content_result['total_batches']} батчей)"
)

[batch_get_content_start] Загрузка контента для 300 отелей...
[batch_get_content_done] Загружен контент для 300 из 300 отелей (3 батчей)


In [13]:
content_data = []
for hid, content in content_map.items():
    content_data.append({
        "hid": hid,
        "name": content.get("name", "")[:40],
        "stars": content.get("star_rating", 0),
        "kind": content.get("kind", ""),
        "address": content.get("address", "")[:50],
        "latitude": content.get("latitude"),
        "longitude": content.get("longitude"),
        "check_in": content.get("check_in_time", ""),
        "check_out": content.get("check_out_time", ""),
    })

df_content = pd.DataFrame(content_data)

# Merge with prices from df_hotels
df_full = df_hotels.merge(df_content, on="hid", how="left")
df_full = df_full[["hotel_id", "name", "stars", "kind", "price", "meal", "address"]]
df_full = df_full.sort_values("price", ascending=True).reset_index(drop=True)
df_full.index += 1

df_full

,hotel_id,name,stars,kind,price,meal,address
1,na_baumanskoy_hotel,Отель на Бауманской,0,Hotel,6074.0,nomeal,"улица Фридриха Энгельса, д.31/35, Москва"
2,travel_inn_krasnyie_vorota,Мини-отель на Комсомольской,0,Mini-hotel,6951.0,nomeal,"улица Новая Басманная, д.10/1, Москва"
3,apartamentyi_sadovoe_koltso_izumrudnaya,Апартаменты Садовое Кольцо Изумрудная,0,Apartment,6970.0,nomeal,"Изумрудная улица, д.6, Москва"
4,zvezda_hotel_6,Городской отель Звезда,0,Hotel,7000.0,nomeal,"г. Люберцы ул. 3-е Почтовое Отделение, д. 44А,..."
5,minigostinitsa_brusnika_polezhaevskaya,Мини-Отель Брусника Полежаевская,0,Mini-hotel,7149.0,nomeal,"Хорошевское шоссе, д.23к2 помещ.1/1, Москва"
...,...,...,...,...,...,...,...
296,moscow_marriott_hotel_novy_arbat,Отель Звёзды Арбата (ранее Мариотт Новый,5,Hotel,38000.0,nomeal,"ул. Новый Арбат, 32, Москва, Москва"
297,russkie_sezonyi,Бутик-Отель Регул,5,Boutique_and_Design,38106.0,nomeal,"Дегтярный переулок, д. 8/2, Москва"
298,hampton_by_hilton_moscow_rogozhsky_val,Отель Hampton by Hilton Moscow Рогожский,4,Hotel,38522.0,breakfast,"улица Рогожский Вал, дом 10, Москва"
299,grand_revival_hotel,Отель Revival Hotel,3,Hotel,38836.0,nomeal,"улица Петровка 19 стр.3, Москва"


In [14]:
from services import batch_get_reviews, filter_reviews, HotelReviewsFiltered

print(f"[batch_get_reviews_start] Загрузка отзывов для {len(hotel_hids)} отелей...")
raw_reviews = await batch_get_reviews(client, hotel_hids, LANGUAGE)
reviews_map = filter_reviews(raw_reviews)

total_raw = sum(len(revs) for revs in raw_reviews.values())
total_filtered = sum(len(rd["reviews"]) for rd in reviews_map.values())
total_filtered_by_age = sum(rd["filtered_by_age"] for rd in reviews_map.values())
hotels_with_reviews = len(reviews_map)

# Calculate average rating across all hotels
all_avg_ratings = [rd["avg_rating"] for rd in reviews_map.values() if rd["avg_rating"] is not None]
overall_avg = sum(all_avg_ratings) / len(all_avg_ratings) if all_avg_ratings else 0

print(
    f"[batch_get_reviews_done] Всего {hotels_with_reviews} отелей с отзывами из {len(hotel_hids)}"
)
print(
    f"  Обработано {total_raw} отзывов → {total_filtered} релевантных "
    f"(отсечено по давности: {total_filtered_by_age})"
)
print(f"  Средний рейтинг: {overall_avg:.1f}/10")

[batch_get_reviews_start] Загрузка отзывов для 300 отелей...
[batch_get_reviews_done] Всего 297 отелей с отзывами из 300
  Обработано 46975 отзывов → 11792 релевантных (отсечено по давности: 7125)
  Средний рейтинг: 8.5/10


In [15]:
# Create DataFrame with reviews summary
reviews_data = []
for hid, data in reviews_map.items():
    hotel_id = next((h["id"] for h in hotels if h["hid"] == hid), "")
    detailed = data["detailed_averages"]
    reviews_data.append({
        "hotel_id": hotel_id,
        "hid": hid,
        "total": data["total_reviews"],
        "avg_rating": data["avg_rating"],
        "cleanness": detailed["cleanness"],
        "location": detailed["location"],
        "price": detailed["price"],
        "services": detailed["services"],
        "room": detailed["room"],
        "meal": detailed["meal"],
        "wifi": detailed["wifi"],
        "hygiene": detailed["hygiene"],
    })

df_reviews = pd.DataFrame(reviews_data)
df_reviews = df_reviews.sort_values("avg_rating", ascending=False, na_position="last").reset_index(drop=True)
df_reviews.index += 1

# Merge with hotel info
df_reviews_full = df_reviews.merge(
    df_content[["hid", "name", "stars"]],
    on="hid",
    how="left"
)
df_reviews_full = df_reviews_full[[
    "hotel_id", "name", "stars", "total", "avg_rating",
    "cleanness", "location", "room", "services", "price", "meal", "wifi", "hygiene"
]]


def show_reviews(hotel_id: str, limit: int = 5) -> None:
    """Show reviews for a hotel."""
    hid = next((h["hid"] for h in hotels if h["id"] == hotel_id), None)
    if not hid:
        print(f"Hotel '{hotel_id}' not found")
        return
    
    data = reviews_map.get(hid)
    if not data:
        print(f"No reviews for hotel '{hotel_id}'")
        return
    
    hotel_name = content_map.get(hid, {}).get("name", hotel_id)
    avg = data["avg_rating"]
    d = data["detailed_averages"]
    
    print(f"{'='*70}")
    print(f"{hotel_name}")
    print(f"Total: {data['total_reviews']} reviews | Avg rating: {avg}/10" if avg else f"Total: {data['total_reviews']} reviews")
    print(f"{'='*70}")
    print(f"Detailed scores:")
    print(f"  cleanness: {d['cleanness']}  location: {d['location']}  room: {d['room']}  services: {d['services']}")
    print(f"  price: {d['price']}  meal: {d['meal']}  wifi: {d['wifi']}  hygiene: {d['hygiene']}")
    print(f"{'='*70}\n")
    
    reviews = data["reviews"][:limit]
    for r in reviews:
        rating = r["rating"]
        date = r["created"][:10]
        lang = r.get("_lang", "?")
        plus = r.get("review_plus", "").strip()
        minus = r.get("review_minus", "").strip()
        print(f"[{rating}/10] {date} [{lang}]")
        if plus:
            print(f"  + {plus[:300]}")
        if minus:
            print(f"  - {minus[:300]}")
        print()


df_reviews_full

,hotel_id,name,stars,total,avg_rating,cleanness,location,room,services,price,meal,wifi,hygiene
0,sovremennaya_i_stilnaya_kvartira_ryadom_s_kremlem,Квартира GM Apartments рядом с Кремлем н,0,3,10.0,10.0,10.0,10.0,10.0,10.0,NaN,10.0,NaN
1,great_apart_na_arbate_1,Great apart Duplex на Арбате,0,1,10.0,10.0,10.0,10.0,10.0,10.0,NaN,10.0,NaN
2,bobolink_cold_na_sokolinoy_gore_lodging_house,Апартаменты двухуровневые Cosmo Star на,0,9,10.0,10.0,10.0,10.0,10.0,10.0,NaN,10.0,NaN
3,volgogradskiy_pospekt_325_k3_apartments,Апартаменты Волгоградский Проспект 32/5,0,7,10.0,10.0,10.0,10.0,10.0,10.0,NaN,10.0,NaN
4,vmestimostyyu_do_5_gostey_ryadom_s_metro_ot_ts...,Квартира Вместимостью до 5 Гостей Рядом,0,1,10.0,10.0,10.0,10.0,10.0,10.0,NaN,10.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,ladomir_v_filyah,Отель Мини-отель Ладомир-отель,0,26,4.9,4.5,5.6,3.9,5.1,5.2,2.6,10.0,2.0
293,zvezda_hotel_6,Городской отель Звезда,0,96,4.8,4.6,5.1,4.0,5.0,4.9,6.2,10.0,5.0
294,hotel_apelsin_vnukovo,Апельсин Гостевой Дом во Внуково,0,35,4.4,4.1,5.9,3.6,4.0,3.8,4.8,10.0,NaN
295,moskva_siti_apex_42_etazh_2_komnaty_apartments,Апартаменты Москва Сити Apex 42 этаж 2 к,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# Example: view reviews for a specific hotel
# show_reviews("four_seasons_st_petersburg")       # 5 reviews
# show_reviews("four_seasons_st_petersburg", 10)   # 10 reviews

show_reviews("four_seasons_st_petersburg", limit=3)

Hotel 'four_seasons_st_petersburg' not found


In [17]:
from services import combine_hotels_data, HotelFull

combined = combine_hotels_data(hotels, content_map, reviews_map)
print(f"Combined {len(combined)} hotels with content and reviews")

Combined 300 hotels with content and reviews


In [18]:
import json

from services import estimate_tokens, prepare_hotel_for_llm, presort_hotels, score_hotels

# Estimate tokens before presort
hotels_for_llm_all = [prepare_hotel_for_llm(h) for h in combined]
tokens_before = estimate_tokens(json.dumps(hotels_for_llm_all, ensure_ascii=False))

# Pre-sort by hotel kind tier and prescore, limit to top 100 for LLM scoring
PRESORT_LIMIT = 300
top_hotels = presort_hotels(combined, reviews_map, limit=PRESORT_LIMIT)

# Estimate tokens after presort
hotels_for_llm_top = [prepare_hotel_for_llm(h) for h in top_hotels]
tokens_after = estimate_tokens(json.dumps(hotels_for_llm_top, ensure_ascii=False))

print(f"[presort_done] {len(combined)} отелей → {len(top_hotels)} (лимит {PRESORT_LIMIT})")
print(f"  Токены: ~{tokens_before:,} → ~{tokens_after:,} (экономия {tokens_before - tokens_after:,})")

[presort_done] 300 отелей → 300 (лимит 300)
  Токены: ~329,260 → ~329,260 (экономия 0)


In [19]:
import time

from services import score_hotels

# Score hotels using single LLM request
# Returns top 10 scored hotels with summary explaining the selection
print(f"[scoring_start] Scoring {len(top_hotels)} hotels...")
start_time = time.time()

scoring_result = await score_hotels(
    top_hotels,
    USER_PREFERENCES,
    guests=GUESTS,
    min_price=MIN_PRICE,
    max_price=MAX_PRICE,
    currency=CURRENCY,
)

elapsed = time.time() - start_time

if scoring_result["error"]:
    print(f"\n❌ ERROR: {scoring_result['error']}")
    scoring_results = None
else:
    scoring_results = scoring_result["results"]
    print(f"[scoring_done] {len(scoring_results)} hotels scored — {elapsed:.1f}s")
    print(f"  Estimated tokens: ~{scoring_result['estimated_tokens']:,}")
    print(f"\n📋 Summary:\n{scoring_result['summary']}")

[scoring_start] Scoring 300 hotels...
[scoring_done] 10 hotels scored — 35.1s
  Estimated tokens: ~330,711

📋 Summary:
The Moscow market currently offers excellent 4-star options within the 20,000 RUB budget, allowing us to prioritize Tier 1 and Tier 2 properties with high guest ratings. We specifically selected hotels like 'Salyut Apart' (ID: salyut_aparthotel, 9.8/10) and 'Kunlun' (ID: kunlun, 8.6/10) because they offer the requested multi-room layouts and high cleanliness standards. We rejected several cheaper options due to the 'Anti-Downgrade' rule; for example, 'Wals Hotel' (ID: wals_hotel) was excluded due to its low 2-star status and poor 6.0/10 rating, while 'Hotel Soblaznov' (ID: hotel_soblaznov) was disqualified as a 'trap' with a 5.4/10 rating despite its 3-star claim. The final selection focuses on properties rated 8.5 or higher to ensure the user's preference for cleanliness and quality is met.


In [21]:
from typing import Any


def display_top_hotels(
    results: list[dict[str, Any]],
    hotels_data: list[dict[str, Any]],
    top_n: int = 10,
) -> pd.DataFrame:
    """Display top N scored hotels with details and Ostrovok links."""
    # Build lookup maps
    name_map = {h.get("id", ""): h.get("name", h.get("id", "")) for h in hotels_data}
    hid_map = {h.get("id", ""): h.get("hid", 0) for h in hotels_data}
    reviews_map_by_id = {h.get("id", ""): h.get("reviews", {}) for h in hotels_data}
    
    print(f"\n{'='*80}")
    print(f"TOP {top_n} HOTELS")
    print(f"{'='*80}\n")

    data = []
    for i, hotel in enumerate(results[:top_n], 1):
        hotel_id = hotel.get("hotel_id", "")
        score = hotel.get("score", 0)
        name = name_map.get(hotel_id, hotel_id)
        hid = hid_map.get(hotel_id, 0)
        reasons = hotel.get("top_reasons", [])
        penalties = hotel.get("score_penalties", [])
        
        # Get reviews data
        reviews_data = reviews_map_by_id.get(hotel_id, {})
        avg_rating = reviews_data.get("avg_rating")
        detailed = reviews_data.get("detailed_averages", {})
        
        # Generate Ostrovok URL
        url = ostrovok_url(
            hotel_id=hotel_id,
            hid=hid,
            checkin=CHECKIN_DATE,
            checkout=CHECKOUT_DATE,
            guests=GUESTS,
            region_id=region_id,
        )

        # Print detailed info
        print(f"{i}. {name}")
        print(f"   Score: {score}/100 | Rating: {avg_rating}/10" if avg_rating else f"   Score: {score}/100")
        if reasons:
            print(f"   + {'; '.join(reasons[:3])}")
        if penalties:
            print(f"   - {'; '.join(penalties[:5])}")
        print(f"   🔗 {url}")
        print()
        
        # Collect for DataFrame
        data.append({
            "name": name[:35],
            "score": score,
            "rating": avg_rating,
            "clean": detailed.get("cleanness"),
            "loc": detailed.get("location"),
            "room": detailed.get("room"),
            "svc": detailed.get("services"),
            "price": detailed.get("price"),
            "url": url,
        })
    
    df = pd.DataFrame(data)
    df.index = range(1, len(df) + 1)
    total_found = len(hotels_data)
    selected = min(top_n, len(results))
    print(f"Всего найдено {total_found} отелей на эти даты. ")
    print(f"Подобраны лучшие {selected} по вашим критериям.")
    return df


pd.set_option("display.max_colwidth", 100)
display_top_hotels(scoring_results, combined, top_n=10)


TOP 10 HOTELS

1. Отель Ваш Салют Апарт
   Score: 98/100 | Rating: 8.6/10
   + Premium 4-Star Apart-hotel (Tier 2) within budget; Exceptional guest rating (9.8/10) confirms cleanliness and quality; Spacious Family Apartment with kitchen ideal for 2 adults and 2 children
   🔗 https://ostrovok.ru/hotel/russia/moscow/mid11176521/salyut_aparthotel/?dates=02.02.2026-04.02.2026&guests=4&q=2395

2. Отель Кунь Лунь
   Score: 95/100 | Rating: 8.8/10
   + 4-Star Hotel (Tier 1) offering a specific 2-room Suite; High guest rating (8.6/10) for a very competitive price; Guaranteed two-room layout as requested by user
   - Location is 19km from center
   🔗 https://ostrovok.ru/hotel/russia/moscow/mid10013909/kunlun/?dates=02.02.2026-04.02.2026&guests=4&q=2395

3. Апарт-отель Idera
   Score: 94/100 | Rating: 9.3/10
   + 4-Star Apart-hotel (Tier 2) with high rating (9.3/10); Modern Duplex room provides the requested space separation; Excellent value for a high-tier property
   - Duplex layout might be 

,name,score,rating,clean,loc,room,svc,price,url
1,Отель Ваш Салют Апарт,98,8.6,8.6,8.8,8.7,8.3,8.7,https://ostrovok.ru/hotel/russia/moscow/mid11176521/salyut_aparthotel/?dates=02.02.2026-04.02.20...
2,Отель Кунь Лунь,95,8.8,9.2,8.5,8.9,8.9,9.2,https://ostrovok.ru/hotel/russia/moscow/mid10013909/kunlun/?dates=02.02.2026-04.02.2026&guests=4...
3,Апарт-отель Idera,94,9.3,9.6,9.3,9.4,9.5,9.1,https://ostrovok.ru/hotel/russia/moscow/mid13106596/idera_hotel/?dates=02.02.2026-04.02.2026&gue...
4,Отель WineWood Moscow,92,8.9,8.9,9.3,8.4,9.2,8.3,https://ostrovok.ru/hotel/russia/moscow/mid8854262/winewood_moscow_lounge_hotel/?dates=02.02.202...
5,Отель Авион Внуково,90,9.2,9.4,8.8,9.3,9.3,9.3,https://ostrovok.ru/hotel/russia/moscow/mid6426482/vnukovo_otel_green_palace/?dates=02.02.2026-0...
6,Отель Максима Панорама,89,9.0,9.1,9.6,8.8,8.8,8.9,https://ostrovok.ru/hotel/russia/moscow/mid7467382/maxima_panorama_hotel/?dates=02.02.2026-04.02...
7,Отель Райкин Плаза,88,8.9,9.1,9.2,8.4,8.9,8.8,https://ostrovok.ru/hotel/russia/moscow/mid7930655/raikin_plaza_hotel/?dates=02.02.2026-04.02.20...
8,Отель Spektrcity Taganskaya,87,9.1,9.1,9.3,9.0,9.1,9.0,https://ostrovok.ru/hotel/russia/moscow/mid11024207/spektrcity_taganskaya_hotel/?dates=02.02.202...
9,Отель Альянс Бородино,86,8.5,8.7,8.4,7.9,8.4,8.4,https://ostrovok.ru/hotel/russia/moscow/mid7599836/borodino_hotel/?dates=02.02.2026-04.02.2026&g...
10,Отель East Gate,85,8.2,7.9,9.0,7.6,8.2,7.8,https://ostrovok.ru/hotel/russia/moscow/mid7778713/east_gate_hotel/?dates=02.02.2026-04.02.2026&...
